In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [1]:
import sklearn
import os
from sklearn.linear_model import LinearRegression
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.base import RegressorMixin
from scipy import fftpack
from sklearn.preprocessing import PolynomialFeatures
from statsmodels.tsa.stattools import adfuller

c:\users\pratik doshi\appdata\local\programs\python\python37\lib\site-packages\statsmodels\compat\pandas.py:49: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  data_klasses = (pandas.Series, pandas.DataFrame, pandas.Panel)


# Fetching Relevant Data

The for a sample of sectoral indices have been taken. The sample is selected purely based on availability. Only those indices that have data for more than 2000 days (8 years) have been used. The list has been compiled in PB Analysis Notebook.

In [2]:
sample = ['Nifty Auto',
 'Nifty Bank',
 'Nifty FMCG',
 'Nifty Media',
 'Nifty Midcap 50',
 'Nifty',
 'Nifty IT',
 'Nifty Pharma',
 'Nifty Realty']

In [4]:
base_dir = '.\\Data\\'

def load_df(folder):
    return pd.read_csv(folder,usecols = [1,2,3,4],parse_dates = ['Date']).sort_values(by='Date').reset_index()

refer = {}
i = 0
for file in sample:
    print(file)
    
    refer[file] = load_df(base_dir + file + '.csv')
    print()


Nifty Auto

Nifty Bank

Nifty FMCG

Nifty Media

Nifty Midcap 50

Nifty

Nifty IT

Nifty Pharma

Nifty Realty



# Splitting Training and Testing Data

The process will involve fitting a simple time series regression. Adding the seasonality component to the residuals of the regression and analysing the residuals that remain after adjusting for drift and seasonality. The P/B Analysis Notebook has already established that none of the indices have a stationary P/B ratio. The graphs of the indices however show little to no trend. The trend if exists will reflect in the time series regression.

In [5]:
# Splitting training and testing data
def split(X,test_size = 0.1):
    classify = X.index < int(len(X)*(1-test_size))
    
    X_train = X[classify]
    X_test = X[~classify]
    
    return X_train, X_test

test = {}
train = {}
for e in refer.keys():
    train[e], test[e] = split(refer[e])
    print(e,'; Training Size: ' + str(len(train[e])),'; Testing Size: ' + str(len(test[e])))

Nifty Auto ; Training Size: 3151 ; Testing Size: 351
Nifty Bank ; Training Size: 4473 ; Testing Size: 497
Nifty FMCG ; Training Size: 2074 ; Testing Size: 231
Nifty Media ; Training Size: 3152 ; Testing Size: 351
Nifty Midcap 50 ; Training Size: 2329 ; Testing Size: 259
Nifty ; Training Size: 4473 ; Testing Size: 497
Nifty IT ; Training Size: 4465 ; Testing Size: 497
Nifty Pharma ; Training Size: 2074 ; Testing Size: 231
Nifty Realty ; Training Size: 2982 ; Testing Size: 332


# Time Series Regression Model

In [6]:
#Building transformer to use the indices as the independent variable
class IndexSelector(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        indices = X.index
        return np.array(indices.values.reshape(-1,1))

In [7]:
from ipywidgets import interact
#Modelling Drift
drift = {}
for index in train.keys():
    drift[index] = Pipeline([('IndexSelector',IndexSelector()),
                            ('Regressor',LinearRegression())])
    drift[index].fit(train[index],train[index]['P/B'])

def plot_model(feature):
    print('R2 for testing data:',drift[feature].score(test[feature],test[feature]['P/B']))
    predicted = drift[feature].predict(train[feature])
    plt.figure(figsize = (10,15))
    
    plt.subplot(3,1,1)
    plt.title(feature + ': Linear Regression')
    plt.plot(train[feature]['Date'],train[feature]['P/B'], label = 'Actual P/B Ratio')
    plt.plot(train[feature]['Date'],predicted,label = 'Predicted P/B Ratio')
    plt.legend()

    
    plt.subplot(3,1,2)
    plt.title(feature + ': Regression Residuals')
    plt.plot(train[feature]['Date'],train[feature]['P/B'] - predicted, label ='Residuals')
    plt.legend()
    
    
    plt.subplot(3,1,3)
    plt.title(feature + ': Regression Performance on Test Data')
    plt.plot(test[feature]['Date'],test[feature]['P/B'], label = 'P/B Ratio')
    plt.plot(test[feature]['Date'],drift[feature].predict(test[feature]), label = 'Test Predictions')
    plt.legend()
#    plt.savefig('Z:\\Research\\Book Value per Share Analysis\\Project Material\\Charts\\' + feature 
#              + '_LinearModel3.png')
    
    
    
interact(plot_model, feature = train.keys())
#for index in train.keys():
#    plot_model(index)

interactive(children=(Dropdown(description='feature', options=('Nifty Auto', 'Nifty Bank', 'Nifty FMCG', 'Nift…

<function __main__.plot_model(feature)>

The R2 values on the testing data is very poor for linear regression. A custom regressor that simply returns the mean will be a better model than a linear regression.

# Naive Mean Predicting Model

In [8]:
#CustomRegressor
class MeanRegressor(BaseEstimator, RegressorMixin):
    def __init__(self):
        self.mean = 0
    
    def fit(self, X, y=None):
        self.mean = np.median(y, axis = 0)
    
    def predict(self, X, y=None):
        pred = np.zeros(X.shape[0])
        pred[0:] = self.mean
        return pred

In [9]:
#Refitting drift model using meanregressor
from ipywidgets import interact
mean_drift = {}
for index in train.keys():
    mean_drift[index] = Pipeline([('IndexSelector',IndexSelector()),
                            ('Regressor',MeanRegressor())])
    mean_drift[index].fit(train[index],train[index]['P/B'])

def plot_model(feature):
    print('R2 for testing data:',mean_drift[feature].score(test[feature],test[feature]['P/B']))
    predicted = mean_drift[feature].predict(train[feature])
    plt.figure(figsize = (10,15))
    
    plt.subplot(3,1,1)
    plt.title(feature + ': Naive Mean Model')
    plt.plot(train[feature]['Date'],train[feature]['P/B'], label = 'Actual P/B Ratio')
    plt.plot(train[feature]['Date'],predicted,label = 'Predicted P/B Ratio')
    plt.legend()
    
    plt.subplot(3,1,2)
    plt.title(feature + ': Mean Model Residuals')
    plt.plot(train[feature]['Date'],train[feature]['P/B'] - predicted, label ='Residuals')
    plt.legend()
    
    plt.subplot(3,1,3)
    plt.title(feature + ': Performance of Predicting Mean on Test Data')
    plt.plot(test[feature]['Date'],test[feature]['P/B'], label = 'P/B Ratio')
    plt.plot(test[feature]['Date'],mean_drift[feature].predict(test[feature]), label = 'Test Predictions')
    plt.legend()
    
#    plt.savefig('Z:\\Research\\Book Value per Share Analysis\\Project Material\\Charts\\' + feature 
#              + '_MeanModel3.png')
    
interact(plot_model, feature = train.keys())
#for index in train.keys():
#    plot_model(index)

interactive(children=(Dropdown(description='feature', options=('Nifty Auto', 'Nifty Bank', 'Nifty FMCG', 'Nift…

<function __main__.plot_model(feature)>

# Seasonality Componenets with Discrete Fourier Transform

In [12]:
seasonality = {}
frequencies = {}


for index in train.keys():
    seasonality[index] = fftpack.fft(train[index]['P/B'] - train[index]['P/B'].mean())
    seasonality[index] = seasonality[index][:len(seasonality[index])//2]
    frequencies[index] = np.argsort(-np.abs(seasonality[index]))[:5]
    
def plot_seasons(index):
    Y = seasonality[index]
    plt.plot(np.abs(Y))
    plt.xlabel('frequency')
    print('Max at:',np.argmax(Y))
    print('Mean:',np.abs(Y).mean())
    
interact(plot_seasons, index = train.keys())

interactive(children=(Dropdown(description='index', options=('Nifty Auto', 'Nifty Bank', 'Nifty FMCG', 'Nifty …

<function __main__.plot_seasons(index)>

In [13]:
#Building Transformer for Fourier Features
class FourierFeatures(BaseEstimator, TransformerMixin):
    def __init__(self, freq):
        self.freq = freq
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        Xt = np.zeros((len(X),2*len(self.freq)))
        for i in range(len(self.freq)):
            Xt[:,i] = np.sin(2 * np.pi*self.freq[i]*X).reshape(-1)
            Xt[:,2*i+1] = np.cos(2 * np.pi*self.freq[i]*X).reshape(-1)
        return Xt

In [14]:
class MeanFeature(BaseEstimator, TransformerMixin):
    def __int__(self):
        pass
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        m = X.mean(axis = 1)[0]
        Xt = np.zeros(len(X))
        Xt[:] = m
        return Xt.reshape((-1,1))

In [15]:
seasons = {}

for index in train.keys():
    union = FeatureUnion([('mean',MeanFeature()),('fourier',FourierFeatures(frequencies[index]))])
    seasons[index] = Pipeline([('Index',IndexSelector()),
                              ('union',union),
                              ('regressor',LinearRegression())])
    seasons[index].fit(train[index],train[index]['P/B'])

In [16]:
#Evaluation model with seasonality component
from ipywidgets import interact


def plot_model(feature):
    print('R2 for testing data:',seasons[feature].score(test[feature],test[feature]['P/B']))
    predicted = seasons[feature].predict(train[feature])
    plt.figure(figsize = (10,8))
    
    plt.subplot(3,1,1)
    plt.plot(train[feature]['Date'],train[feature]['P/B'], label = 'Actual P/B')
    plt.plot(train[feature]['Date'],predicted,label = 'Model P/B')
    plt.legend()
    
    plt.subplot(3,1,2)
    plt.plot(train[feature]['Date'],train[feature]['P/B'] - predicted, label ='Residual')
    plt.legend()
    
    plt.subplot(3,1,3)
    plt.plot(test[feature]['Date'],test[feature]['P/B'], label = 'Test P/B')

    plt.plot(test[feature]['Date'],seasons[feature].predict(test[feature]), label = 'Test Predictions')
    plt.legend()
    
interact(plot_model, feature = train.keys())

interactive(children=(Dropdown(description='feature', options=('Nifty Auto', 'Nifty Bank', 'Nifty FMCG', 'Nift…

<function __main__.plot_model(feature)>

# Polynomial Regression Model

In [18]:
poly_reg = {}
for index in train.keys():
    poly_reg[index] = []
    for i in range(1,8):
        pipe = Pipeline([('Index',IndexSelector()),
                ('Polynomial Features',PolynomialFeatures(degree = i)),
                ('Regression',LinearRegression())])
        pipe.fit(train[index],train[index]['P/B'])
        poly_reg[index].append(pipe)


In [20]:
from ipywidgets import interact, FloatSlider, IntSlider

def evaluate_model(index, degree):
    degree = degree - 1
    r2 = poly_reg[index][degree].score(test[index],test[index]['P/B'])
    rtrain = poly_reg[index][degree].score(train[index],train[index]['P/B'])
    train_pred = poly_reg[index][degree].predict(train[index])
    test_pred = poly_reg[index][degree].predict(test[index])
    residual = train[index]['P/B'] - train_pred 
    
    print('R2 on Test:',r2)
    print('R2 on Train:',rtrain)
    print('Variance:', rtrain - r2)
    plt.figure(figsize = (10,15))
    
    plt.subplot(3,1,1)
    plt.title(index + ': Polynomial Regression, Degree = ' + str(degree+1))
    plt.plot(train[index]['Date'],train[index]['P/B'],label = 'Actual P/B Ratio')
    plt.plot(train[index]['Date'],train_pred,label = 'Predicted P/B Ratio')
    plt.legend()
    
    plt.subplot(3,1,2)
    plt.title(index + ': Regression Residuals')
    plt.plot(residual,label ='Residuals')
    plt.legend()
    
    plt.subplot(3,1,3)
    plt.title(index + ': Regression Performance on Test Data')
    plt.plot(test[index]['Date'],test[index]['P/B'],label = 'P/B Ratio')
    plt.plot(test[index]['Date'],test_pred, label = 'Test Predictions')
    plt.legend()
    
    #plt.savefig('Z:\\Research\\Book Value per Share Analysis\\Project Material\\Charts\\' + index
    #          + '_PolynomialD' + str(degree + 1) + '.png')

slider = IntSlider(min=1, max=7, step=1, description="Degree")
interact(evaluate_model, index = train.keys(), degree = slider) 
#for index in train.keys():
#    for d in range(1,8):
#        evaluate_model(index,d)

interactive(children=(Dropdown(description='index', options=('Nifty Auto', 'Nifty Bank', 'Nifty FMCG', 'Nifty …

<function __main__.evaluate_model(index, degree)>

In [32]:
#Analysis of R2 in train, R2 in test in different degrees of polynomial regression
r2_train = []
r2_test = []
index_name = []
deg = []
for index in poly_reg.keys():
    for i,d in enumerate(poly_reg[index]):
        deg.append(i+1)
        index_name.append(index)
        r2_train.append(d.score(train[index],train[index]['P/B']))
        r2_test.append(d.score(test[index],test[index]['P/B']))


In [33]:
dic = {'Index':index_name,'Degree of Regression':deg,'R-Square on training data':r2_train,
                                                        'R-Square on testing data':r2_test} 
df = pd.DataFrame(dic)

In [34]:
df.head(10)

,Index,Degree of Regression,R-Square on training data,R-Square on testing data
0,Nifty Auto,1,0.212648,-19.016024
1,Nifty Auto,2,0.361723,-46.610782
2,Nifty Auto,3,0.655984,-1.406340
3,Nifty Auto,4,0.658753,0.080568
4,Nifty Auto,5,0.713320,-45.136930
5,Nifty Auto,6,0.549455,-0.557598
6,Nifty Auto,7,0.525616,-0.093922
7,Nifty Bank,1,0.413588,-0.114996
8,Nifty Bank,2,0.505744,-2.171626
9,Nifty Bank,3,0.560017,-0.608056


In [35]:
df['Model Variance'] = df['R-Square on training data'] - df['R-Square on testing data']

In [36]:
df.to_csv(r'Z:\Research\Book Value per Share Analysis\Project Material\Polynomial Regression Score.csv', index= False)

# Last N Day Average Model

In [21]:
class MovingAverageRegressor(BaseEstimator, RegressorMixin):
    def __init__(self, n):
        self.N = n
        self.NAvg = 0
    def fit(self, X, y):
        self.NAvg = y[-self.N:].mean()
        return self
    def predict(self, X):
        pred = np.zeros(len(X))
        pred[:] = self.NAvg
        return pred

In [22]:
N_Periods = [50,100,150,200]
mavg = {}
for index in train.keys():
    mavg[index] = []
    for period in N_Periods:
        reg = MovingAverageRegressor(period)
        reg.fit(X = None, y = train[index]['P/B'])
        mavg[index].append(reg)

In [23]:
from ipywidgets import interact

def final_plot(index, period):
    period = N_Periods.index(period)
    r2_mavg = mavg[index][period].score(test[index],test[index]['P/B'])
    r2_reg = drift[index].score(test[index],test[index]['P/B'])
    r2_mean = mean_drift[index].score(test[index],test[index]['P/B'])
    train_pred = mavg[index][period].predict(train[index])
    test_pred = mavg[index][period].predict(test[index])
    residual = train[index]['P/B'] - train_pred
    print('Moving Average Reg:',r2_mavg)
    print('Linear Reg:',r2_reg)
    print('Mean Reg:',r2_mean)
    
    plt.figure(figsize = (10,15))
    period = N_Periods[period]
    plt.subplot(3,1,1)
    plt.title(index + ': ' + str(period) + ' Day Average Predicting Model')
    plt.plot(train[index]['Date'],train[index]['P/B'],label = 'Actual P/B Ratio')
    plt.plot(train[index]['Date'],train_pred,label = 'Predicted P/B Ratio')
    plt.legend()
    
    plt.subplot(3,1,2)
    plt.title(index + ': ' + str(period) + ' Day Average Residuals')
    plt.plot(residual,label ='Residuals')
    plt.legend()
    
    plt.subplot(3,1,3)
    plt.title(index + ': ' + str(period) + ' Day Average Performance on Test Data')
    plt.plot(test[index]['Date'],test[index]['P/B'],label = 'Test P/B')
    plt.plot(test[index]['Date'],test_pred, label = 'Model')
    plt.legend()
#    plt.savefig('Z:\\Research\\Book Value per Share Analysis\\Project Material\\Charts\\' + index 
#              + str(period) +  'DayAvg.png')

interact(final_plot, index = train.keys(), period = N_Periods)   
#for index in train.keys():
#    for p in N_Periods:
#        final_plot(index,p)

interactive(children=(Dropdown(description='index', options=('Nifty Auto', 'Nifty Bank', 'Nifty FMCG', 'Nifty …

<function __main__.final_plot(index, period)>

# Autocorrelation Analysis

In [24]:
from pandas.plotting import autocorrelation_plot
from ipywidgets import interact

def plot(feature):
    plt.figure(figsize = (10,6))
    a = autocorrelation_plot(refer[feature]['P/B'])
    plt.xlabel('Lag (Days)')
    plt.savefig('Z:\\Research\\Book Value per Share Analysis\\Project Material\\Charts\\' + feature 
              + '_Correl.png')
interact(plot, feature = refer.keys())
#for index in train.keys():
#    plot(index)

interactive(children=(Dropdown(description='feature', options=('Nifty Auto', 'Nifty Bank', 'Nifty FMCG', 'Nift…

<function __main__.plot(feature)>

R2 from a regressor that uses something like moving average vs R2 from mean and linear regressors. Use autocorrelation

# Saving Performance Data of Linear Regression, Naive Mean Model and Last N Day Average Model

In [41]:
#Comparing the performance of the regressions: test
mean_perf = []
reg_perf = []
avg50_perf = []
avg100_perf = []
avg150_perf = []
avg200_perf = []
for feature in test.keys():
    mean_perf.append(mean_drift[feature].score(test[feature],test[feature]['P/B']))
    reg_perf.append(drift[feature].score(test[feature],test[feature]['P/B']))
    avg50_perf.append(mavg[feature][0].score(test[feature],test[feature]['P/B']))
    avg100_perf.append(mavg[feature][1].score(test[feature],test[feature]['P/B']))
    avg150_perf.append(mavg[feature][2].score(test[feature],test[feature]['P/B']))
    avg200_perf.append(mavg[feature][3].score(test[feature],test[feature]['P/B']))

comparison = pd.DataFrame({'Index':list(test.keys()),'Mean Regression':mean_perf,'Linear Regression':reg_perf,
                          '50 Day MA':avg50_perf,'100 Day MA':avg100_perf,'150 Day MA':avg150_perf,
                          '200 Day MA':avg200_perf})
comparison.to_csv('Z:\\Research\\Book Value per Share Analysis\\R2TestDataComparison.csv', index= False)

In [42]:
#Comparing the performance of the regressions: train
mean_perf = []
reg_perf = []
avg50_perf = []
avg100_perf = []
avg150_perf = []
avg200_perf = []
for feature in test.keys():
    mean_perf.append(mean_drift[feature].score(train[feature],train[feature]['P/B']))
    reg_perf.append(drift[feature].score(train[feature],train[feature]['P/B']))
    avg50_perf.append(mavg[feature][0].score(train[feature],train[feature]['P/B']))
    avg100_perf.append(mavg[feature][1].score(train[feature],train[feature]['P/B']))
    avg150_perf.append(mavg[feature][2].score(train[feature],train[feature]['P/B']))
    avg200_perf.append(mavg[feature][3].score(train[feature],train[feature]['P/B']))

comparison = pd.DataFrame({'Index':list(test.keys()),'Mean Regression':mean_perf,'Linear Regression':reg_perf,
                          '50 Day MA':avg50_perf,'100 Day MA':avg100_perf,'150 Day MA':avg150_perf,
                          '200 Day MA':avg200_perf})
comparison.to_csv('Z:\\Research\\Book Value per Share Analysis\\R2TrainDataComparison.csv', index= False)

# Stationarity

In [26]:
# Saving Results of ADF Test
def station(index):
    result = adfuller(refer[index]['P/B'])
    
    plt.figure(figsize = (10,6))
    plt.title(index)
    plt.plot(refer[index]['Date'],refer[index]['P/B'], label = 'P/B Ratio')
    plt.plot(refer[index]['Date'],np.full(len(refer[index]['Date']),refer[index]['P/B'].mean()), label = ' Mean P/B Ratio')
    plt.legend()
#    plt.savefig('Z:\\Research\\Book Value per Share Analysis\\Project Material\\Charts\\' + index 
#              + '_Stat.png')
    
    return result[1]

p = []
for index in refer.keys():
    p.append(station(index))
pd.DataFrame({'Index':list(refer.keys()),'P-Value':p}).to_csv('Z:\\Research\\Book Value per Share Analysis\\Project Material\\Stationarity.csv')

In [29]:
#cummulative moving averae and cummulative moving variance

from ipywidgets import interact

def getVariance(series):
    l1 = np.array([series.iloc[:i].var() for i in range(len(series))])
    l2 = np.array([series.iloc[:i].mean() for i in range(len(series))])
    return l1,l2


#Comparing given data to a generated stationary series with a comparable mean and variance
def stat_analysis(index):
    stationary_series = np.random.normal(loc = refer[index]['P/B'].mean(), scale =refer[index]['P/B'].std(), size = len(refer[index]))
    plt.figure(figsize = (20,20))

    plt.subplot(3,2,1)
    plt.title(index)
    plt.plot(refer[index]['Date'],refer[index]['P/B'], label = 'P/B Ratio')
    plt.plot(refer[index]['Date'], np.full(len(refer[index]['P/B']),refer[index]['P/B'].mean()), label = 'Mean P/B Ratio')
    plt.legend()


    plt.subplot(3,2,2)
    plt.title('Comparative Stationary Series')
    plt.plot(refer[index]['Date'], stationary_series, label = 'Stationary Series')
    plt.plot(refer[index]['Date'], np.full(len(stationary_series),stationary_series.mean()), label = 'Mean')
    plt.legend()

    stat = getVariance(pd.Series(stationary_series))
    it = getVariance(pd.Series(refer[index]['P/B']))

    plt.subplot(3,2,3)
    plt.title(index + ': Cummulative Mean')
    plt.plot(it[1], label = 'Cummulative Mean')
    plt.legend()

    plt.subplot(3,2,4)
    plt.title('Comparative Stationary Series: Cummulative Mean')
    plt.plot(stat[1],label = 'Cummulative Mean')
    plt.legend()

    plt.subplot(3,2,5)
    plt.title(index + ': Cummulative Variance')
    plt.plot(it[0],label = 'Cummulative Variance')
    plt.legend()

    plt.subplot(3,2,6)
    plt.title('Comparative Stationary Series: Cummulative Variance')
    plt.plot(stat[0],label = 'Cummulative Variance')
    plt.legend()

    #plt.savefig('Z:\\Research\\Book Value per Share Analysis\\Project Material\\Charts\\' + index 
    #          + '_Stat.png',bbox_inches = 'tight')
    
interact(stat_analysis, index = refer.keys())

interactive(children=(Dropdown(description='index', options=('Nifty Auto', 'Nifty Bank', 'Nifty FMCG', 'Nifty …

<function __main__.stat_analysis(index)>

In [28]:
# Saving figures
#for index in refer.keys():
#    print(index)
#    stat_analysis(index)
#stat_analysis('Nifty')